<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Evaluate RAG with LLM Evals</h1>

In this tutorial we will look into building a RAG pipeline and evaluating it with Phoenix Evals.

It has the the following sections:

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG (with the help of a framework such as LlamaIndex).
3. Evaluating RAG with Phoenix Evals.

## Retrieval Augmented Generation (RAG)

LLMs are trained on vast datasets, but these will not include your specific data (things like company knowledge bases and documentation). Retrieval-Augmented Generation (RAG) addresses this by dynamically incorporating your data as context during the generation process. This is done not by altering the training data of the LLMs but by allowing the model to access and utilize your data in real-time to provide more tailored and contextually relevant responses.

In RAG, your data is loaded and prepared for queries. This process is called indexing. User queries act on this index, which filters your data down to the most relevant context. This context and your query then are sent to the LLM along with a prompt, and the LLM provides a response.

RAG is a critical component for building applications such a chatbots or agents and you will want to know RAG techniques on how to get data into your application.

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/RAG_Pipeline.png">

## Stages within RAG

There are five key stages within RAG, which will in turn be a part of any larger RAG application.

- **Loading**: This refers to getting your data from where it lives - whether it's text files, PDFs, another website, a database or an API - into your pipeline.
- **Indexing**: This means creating a data structure that allows for querying the data. For LLMs this nearly always means creating vector embeddings, numerical representations of the meaning of your data, as well as numerous other metadata strategies to make it easy to accurately find contextually relevant data.
- **Storing**: Once your data is indexed, you will want to store your index, along with any other metadata, to avoid the need to re-index it.

- **Querying**: For any given indexing strategy there are many ways you can utilize LLMs and data structures to query, including sub-queries, multi-step queries, and hybrid strategies. 
- **Evaluation**: A critical step in any pipeline is checking how effective it is relative to other strategies, or when you make changes. Evaluation provides objective measures on how accurate, faithful, and fast your responses to queries are.


## Build a RAG system 

Now that we have understood the stages of RAG, let's build a pipeline. We will use [LlamaIndex](https://www.llamaindex.ai/) for RAG and [Phoenix Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) for evaluation.


In [1]:
!pip install -qq "arize-phoenix[evals,llama-index]" "llama-index-llms-openai" "openai>=1" gcsfs nest_asyncio 'httpx<0.28'

For this tutorial we will be using OpenAI for creating synthetic data as well as for evaluation. 

In [2]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

import pandas as pd
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

import phoenix as px

During this tutorial, we will capture all the data we need to evaluate our RAG pipeline using Phoenix Tracing. To enable this, simply start the phoenix application and instrument LlamaIndex.

In [4]:
px.launch_app().view()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Enable Phoenix tracing via `LlamaIndexInstrumentor`. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [5]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

tracer_provider = register(endpoint="http://127.0.0.1:6006/v1/traces")
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

### Load Data and Build an Index

Let's use an [essay by Paul Graham](https://www.paulgraham.com/worked.html) to build our RAG pipeline.

In [6]:
import tempfile
from urllib.request import urlretrieve

with tempfile.NamedTemporaryFile() as tf:
    urlretrieve(
        "https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/data/paul_graham/paul_graham_essay.txt",
        tf.name,
    )
    documents = SimpleDirectoryReader(input_files=[tf.name]).load_data()

In [7]:
# Define an LLM
llm = OpenAI(model="gpt-4o")

# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

Build a QueryEngine and start querying.

In [8]:
query_engine = vector_index.as_query_engine()

In [9]:
response_vector = query_engine.query("What did the author do growing up?")

Check the response that you get from the query.

In [10]:
response_vector.response

'The author focused on writing short stories and programming, particularly experimenting with early programming languages like Fortran on an IBM 1401 computer during their time in 9th grade.'

By default LlamaIndex retrieves two similar nodes/ chunks. You can modify that in `vector_index.as_query_engine(similarity_top_k=k)`.

Let's check the text in each of these retrieved nodes.

In [11]:
# First retrieved node
response_vector.source_nodes[0].get_text()

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [12]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"I remember taking the boys to the coast on a sunny day in 2015 and figuring out how to deal with some problem involving continuations while I watched them play in the tide pools. It felt like I was doing life right. I remember that because I was slightly dismayed at how novel it felt. The good news is that I had more moments like this over the next few years.\n\nIn the summer of 2016 we moved to England. We wanted our kids to see what it was like living in another country, and since I was a British citizen by birth, that seemed the obvious choice. We only meant to stay for a year, but we liked it so much that we still live there. So most of Bel was written in England.\n\nIn the fall of 2019, Bel was finally finished. Like McCarthy's original Lisp, it's a spec rather than an implementation, although like McCarthy's Lisp it's a spec expressed as code.\n\nNow that I could write essays again, I wrote a bunch about topics I'd had stacked up. I kept writing essays through 2020, but I also s

Remember that we are using Phoenix Tracing to capture all the data we need to evaluate our RAG pipeline. You can view the traces in the phoenix application.

In [13]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


We can access the traces by directly pulling the spans from the phoenix session.

In [14]:
spans_df = px.Client().get_spans_dataframe()

In [15]:
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

,name,span_kind,attributes.input.value,attributes.retrieval.documents
context.span_id,,,,
5c2a05ecb763bf39,BaseEmbedding.get_text_embedding_batch,EMBEDDING,"{""texts"": [""file_path: /var/folders/lz/dhndbs9...",None
5b62c3300844515e,OpenAIEmbedding._get_query_embedding,EMBEDDING,"{""query"": ""What did the author do growing up?""}",None
a964adb12687a8d3,BaseEmbedding.get_query_embedding,EMBEDDING,"{""query"": ""What did the author do growing up?""}",None
e00e5cba5bf6fe56,VectorIndexRetriever._retrieve,RETRIEVER,"{""query_bundle"": {""query_str"": ""What did the a...",None
b71f132096d8b127,BaseRetriever.retrieve,RETRIEVER,What did the author do growing up?,[{'document.content': 'What I Worked On Febru...


Note that the traces have captured the documents that were retrieved by the query engine. This is nice because it means we can introspect the documents without having to keep track of them ourselves.

In [16]:
spans_with_docs_df = spans_df[spans_df["attributes.retrieval.documents"].notnull()]

In [17]:
spans_with_docs_df[["attributes.input.value", "attributes.retrieval.documents"]].head()

,attributes.input.value,attributes.retrieval.documents
context.span_id,,
b71f132096d8b127,What did the author do growing up?,[{'document.content': 'What I Worked On Febru...


We have built a RAG pipeline and also have instrumented it using Phoenix Tracing. We now need to evaluate it's performance. We can assess our RAG system/query engine using Phoenix's LLM Evals. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

## Evaluation

Evaluation should serve as the primary metric for assessing your RAG application. It determines whether the pipeline will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.

In a RAG system, evaluation focuses on two critical aspects:

- **Retrieval Evaluation**: To assess the accuracy and relevance of the documents that were retrieved
- **Response Evaluation**: Measure the appropriateness of the response generated by the system when the context was provided.

### Generate Question Context Pairs

For the evaluation of a RAG system, it's essential to have queries that can fetch the correct context and subsequently generate an appropriate response.

For this tutorial, let's use Phoenix's `llm_generate` to help us create the question-context pairs.

First, let's create a dataframe of all the document chunks that we have indexed.

In [18]:
# Let's construct a dataframe of just the documents that are in our index
document_chunks_df = pd.DataFrame({"text": [node.get_text() for node in nodes]})
document_chunks_df = document_chunks_df.sample(10, random_state=42)
document_chunks_df.head()

,text
0,What I Worked On\n\nFebruary 2021\n\nBefore co...
5,"I applied to 3 grad schools: MIT and Yale, whi..."
46,Surely the biggest source of stress in one's w...
31,"That's what I should have done, just gone off ..."
13,You want to emphasize the visual cues that tel...


Now that we have the document chunks, let's prompt an LLM to generate us 3 questions per chunk. Note that you could manually solicit questions from your team or customers, but this is a quick and easy way to generate a large number of questions.

In [19]:
generate_questions_template = """\
Context information is below.

---------------------
{text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

In [20]:
import json

from phoenix.evals import OpenAIModel, llm_generate


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}


questions_df = llm_generate(
    dataframe=document_chunks_df,
    template=generate_questions_template,
    model=OpenAIModel(model="gpt-3.5-turbo"),
    output_parser=output_parser,
    concurrency=20,
)

llm_generate |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

In [21]:
questions_df.head()

,question_1,question_2,question_3
0,What were the two main things the author worke...,Describe the author's experience working with ...,How did the author's experience with programmi...
5,Explain the author's realization about AI duri...,How did the author's experience with Lisp infl...,Discuss the author's perspective on the field ...
46,What were the two main projects the author was...,What aspects of the job at Y Combinator did th...,Who offered the author unsolicited advice duri...
31,How did the narrator's return to New York impa...,What new approach did the narrator experiment ...,What idea did the narrator have in the spring ...
13,How does the process of painting still lives d...,What was the author's experience like studying...,What did the author learn about software devel...


In [22]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, document_chunks_df], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

The LLM has generated three questions per chunk. Let's take a quick look.

In [23]:
questions_with_document_chunk_df.head(10)

,text,question
0,What I Worked On\n\nFebruary 2021\n\nBefore co...,What were the two main things the author worke...
1,"I applied to 3 grad schools: MIT and Yale, whi...",Explain the author's realization about AI duri...
2,Surely the biggest source of stress in one's w...,What were the two main projects the author was...
3,"That's what I should have done, just gone off ...",How did the narrator's return to New York impa...
4,You want to emphasize the visual cues that tel...,How does the process of painting still lives d...
5,I wrote an essay for myself to answer that que...,How did the author's experience with computers...
6,This name didn't last long before it was repla...,Why did the author decide to name the new comp...
7,"I asked Jessica if she wanted to be president,...",Why did the founders of YC decide to recruit S...
8,Our model turned out to live just down the str...,Explain the difference between painting still ...
9,and went on with the talk. But afterward it oc...,What was the realization that prompted the nar...


### Retrieval Evaluation

We are now prepared to perform our retrieval evaluations. We will execute the queries we generated in the previous step and verify whether or not that the correct context is retrieved.

In [24]:
# loop over the questions and generate the answers
for _, row in questions_with_document_chunk_df.iterrows():
    question = row["question"]
    response_vector = query_engine.query(question)
    print(f"Question: {question}\nAnswer: {response_vector.response}\n")

Question: What were the two main things the author worked on before college?
Answer: The author worked on writing and programming before college.

Question: Explain the author's realization about AI during their first year of grad school. What did they conclude about the traditional way of doing AI?
Answer: The author realized during their first year of grad school that the traditional way of doing AI, which involved programs translating natural language into formal representations and adding them to a list of known things, was flawed. They concluded that these programs only demonstrated a subset of natural language that was a formal language, but a very limited one. The author recognized an unbridgeable gap between what these programs could do and actual understanding of natural language. They believed that simply teaching these programs more words would not solve the fundamental issue with the traditional approach to AI, which relied on explicit data structures representing concepts.

Now that we have executed the queries, we can start validating whether or not the RAG system was able to retrieve the correct context. Let's extract all the retrieved documents from the traces logged to phoenix. (For an in-depth explanation of how to export trace data from the phoenix runtime, consult the [docs](https://docs.arize.com/phoenix/how-to/extract-data-from-spans)).

In [25]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
b71f132096d8b127 0                  e5c50d5ba9af0a6062a8013656052492   
                 1                  e5c50d5ba9af0a6062a8013656052492   
80fad8d2c5defc6c 0                  ad329aa1b860cc37cfff74c79950f182   
                 1                  ad329aa1b860cc37cfff74c79950f182   
7b487cae4015cae5 0                  3e34b82a71ba017613dbd98b05f1666b   
...                                                              ...   
bf9922467897714e 1                  adbe479e47e5a1adc839c5dc04a64277   
7971103777c6c10e 0                  82a0de8041238504fdc9798ceb2c5167   
                 1                  82a0de8041238504fdc9798ceb2c5167   
e35175e968910ed6 0                  e93a2016052d68573a1c4e38810d1446   
                 1                  e93a2016052d68573a1c4e38810d1446   

                                                                                input  \
context.span_id  document_position                                                      
b71f132096d8b127 0                                 What did the author do growing up?   
                 1                                 What did the author do growing up?   
80fad8d2c5defc6c 0                  What were the two main things the author worke...   
                 1                  What were the two main things the author worke...   
7b487cae4015cae5 0                  Explain the author's realization about AI duri...   
...                                                                               ...   
bf9922467897714e 1                  What did the author decide to do after leaving...   
7971103777c6c10e 0                  Discuss the significance of emphasizing visual...   
                 1                  Discuss the significance of emphasizing visual...   
e35175e968910ed6 0                  What was the unique approach taken by the narr...   
                 1                  What was the unique approach taken by the narr...   

                                                                            reference  \
context.span_id  document_position                                                      
b71f132096d8b127 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  I remember taking the boys to the coast on a s...   
80fad8d2c5defc6c 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Even then it took me several years to understa...   
7b487cae4015cae5 0                  I applied to 3 grad schools: MIT and Yale, whi...   
...                                                                               ...   
bf9922467897714e 1                  I asked Jessica if she wanted to be president,...   
7971103777c6c10e 0                  You want to emphasize the visual cues that tel...   
                 1                  Our model turned out to live just down the str...   
e35175e968910ed6 0                  and went on with the talk. But afterward it oc...   
                 1                  The part we got first was to be an angel firm....   

                                    document_score  
context.span_id  document_position                  
b71f132096d8b127 0                        0.814567  
                 1                        0.808837  
80fad8d2c5defc6c 0                        0.841722  
                 1                        0.816228  
7b487cae4015cae5 0                        0.881060  
...                                            ...  
bf9922467897714e 1                        0.826041  
7971103777c6c10e 0                        0.873828  
                 1                        0.861205  
e35175e968910ed6 0                        0.885567  
                 1                        0.872342  

[62 rows x 4 columns]

Let's now use Phoenix's LLM Evals to evaluate the relevance of the retrieved documents with regards to the query. Note, we've turned on `explanations` which prompts the LLM to explain it's reasoning. This can be useful for debugging and for figuring out potential corrective actions.

In [26]:
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
)

relevance_evaluator = RelevanceEvaluator(OpenAIModel(model="gpt-4o"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

run_evals |          | 0/62 (0.0%) | ⏳ 00:00<? | ?it/s

In [27]:
retrieved_documents_relevance_df.head()

label  score  \
context.span_id  document_position                     
b71f132096d8b127 0                   relevant    1.0   
                 1                  unrelated    0.0   
80fad8d2c5defc6c 0                   relevant    1.0   
                 1                  unrelated    0.0   
7b487cae4015cae5 0                   relevant    1.0   

                                                                          explanation  
context.span_id  document_position                                                     
b71f132096d8b127 0                  To determine if the reference text is relevant...  
                 1                  To determine if the reference text contains in...  
80fad8d2c5defc6c 0                  To determine if the reference text is relevant...  
                 1                  To determine if the reference text is relevant...  
7b487cae4015cae5 0                  To determine if the reference text is relevant...

We can now combine the documents with the relevance evaluations to compute retrieval metrics. These metrics will help us understand how well the RAG system is performing.

In [28]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)
documents_with_relevance_df

context.trace_id  \
context.span_id  document_position                                     
b71f132096d8b127 0                  e5c50d5ba9af0a6062a8013656052492   
                 1                  e5c50d5ba9af0a6062a8013656052492   
80fad8d2c5defc6c 0                  ad329aa1b860cc37cfff74c79950f182   
                 1                  ad329aa1b860cc37cfff74c79950f182   
7b487cae4015cae5 0                  3e34b82a71ba017613dbd98b05f1666b   
...                                                              ...   
bf9922467897714e 1                  adbe479e47e5a1adc839c5dc04a64277   
7971103777c6c10e 0                  82a0de8041238504fdc9798ceb2c5167   
                 1                  82a0de8041238504fdc9798ceb2c5167   
e35175e968910ed6 0                  e93a2016052d68573a1c4e38810d1446   
                 1                  e93a2016052d68573a1c4e38810d1446   

                                                                                input  \
context.span_id  document_position                                                      
b71f132096d8b127 0                                 What did the author do growing up?   
                 1                                 What did the author do growing up?   
80fad8d2c5defc6c 0                  What were the two main things the author worke...   
                 1                  What were the two main things the author worke...   
7b487cae4015cae5 0                  Explain the author's realization about AI duri...   
...                                                                               ...   
bf9922467897714e 1                  What did the author decide to do after leaving...   
7971103777c6c10e 0                  Discuss the significance of emphasizing visual...   
                 1                  Discuss the significance of emphasizing visual...   
e35175e968910ed6 0                  What was the unique approach taken by the narr...   
                 1                  What was the unique approach taken by the narr...   

                                                                            reference  \
context.span_id  document_position                                                      
b71f132096d8b127 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  I remember taking the boys to the coast on a s...   
80fad8d2c5defc6c 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Even then it took me several years to understa...   
7b487cae4015cae5 0                  I applied to 3 grad schools: MIT and Yale, whi...   
...                                                                               ...   
bf9922467897714e 1                  I asked Jessica if she wanted to be president,...   
7971103777c6c10e 0                  You want to emphasize the visual cues that tel...   
                 1                  Our model turned out to live just down the str...   
e35175e968910ed6 0                  and went on with the talk. But afterward it oc...   
                 1                  The part we got first was to be an angel firm....   

                                    document_score    eval_label  eval_score  \
context.span_id  document_position                                             
b71f132096d8b127 0                        0.814567      relevant         1.0   
                 1                        0.808837     unrelated         0.0   
80fad8d2c5defc6c 0                        0.841722      relevant         1.0   
                 1                        0.816228     unrelated         0.0   
7b487cae4015cae5 0                        0.881060      relevant         1.0   
...                                            ...           ...         ...   
bf9922467897714e 1                        0.826041      relevant         1.0   
7971103777c6c10e 0                        0.873828      relevant         1.0   

Let's compute Normalized Discounted Cumulative Gain [NCDG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) at 2 for all our retrieval steps.  In information retrieval, this metric is often used to measure effectiveness of search engine algorithms and related applications.

In [29]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.eval_score
    doc_scores[: len(df)] = df.document_score
    try:
        return ndcg_score([eval_scores], [doc_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

In [30]:
ndcg_at_2

,score
context.span_id,
011a023f3b9516b7,0.63093
07ce18ab0582c00c,1.00000
09ab0c33b326f6d1,1.00000
13b993314caadd1e,0.00000
35965650adbfbef9,1.00000
3fdcbd456e1556f7,0.00000
5476b9b10891bf7b,0.00000
5514ebe8d2b98444,1.00000
6102bffe70ec4a1f,1.00000


Let's also compute precision at 2 for all our retrieval steps.

In [31]:
precision_at_2 = pd.DataFrame(
    {
        "score": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) / 2
        )
    }
)

In [32]:
precision_at_2

,score
context.span_id,
011a023f3b9516b7,0.5
07ce18ab0582c00c,0.5
09ab0c33b326f6d1,1.0
13b993314caadd1e,0.0
35965650adbfbef9,1.0
3fdcbd456e1556f7,0.0
5476b9b10891bf7b,0.0
5514ebe8d2b98444,1.0
6102bffe70ec4a1f,1.0


Lastly, let's compute whether or not a correct document was retrieved at all for each query (e.g. a hit)

In [33]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) > 0
        )
    }
)

Let's now view the results in a combined dataframe.

In [34]:
retrievals_df = px.Client().get_spans_dataframe(
    "span_kind == 'RETRIEVER' and input.value is not None"
)
rag_evaluation_dataframe = pd.concat(
    [
        retrievals_df["attributes.input.value"],
        ndcg_at_2.add_prefix("ncdg@2_"),
        precision_at_2.add_prefix("precision@2_"),
        hit,
    ],
    axis=1,
)
rag_evaluation_dataframe

,attributes.input.value,ncdg@2_score,precision@2_score,hit
context.span_id,,,,
e00e5cba5bf6fe56,"{""query_bundle"": {""query_str"": ""What did the a...",NaN,NaN,NaN
b71f132096d8b127,What did the author do growing up?,1.0,0.5,True
40bb2d55d6144028,"{""query_bundle"": {""query_str"": ""What were the ...",NaN,NaN,NaN
80fad8d2c5defc6c,What were the two main things the author worke...,1.0,0.5,True
054d65635e610b75,"{""query_bundle"": {""query_str"": ""Explain the au...",NaN,NaN,NaN
...,...,...,...,...
bf9922467897714e,What did the author decide to do after leaving...,1.0,1.0,True
7b178aba35d355bd,"{""query_bundle"": {""query_str"": ""Discuss the si...",NaN,NaN,NaN
7971103777c6c10e,Discuss the significance of emphasizing visual...,1.0,0.5,True


### Observations

Let's now take our results and aggregate them to get a sense of how well our RAG system is performing.

In [35]:
# Aggregate the scores across the retrievals
results = rag_evaluation_dataframe.mean(numeric_only=True)
results

ncdg@2_score         0.891320
precision@2_score    0.790323
dtype: float64

As we can see from the above numbers, our RAG system is not perfect, there are times when it fails to retrieve the correct context within the first two documents. At other times the correct context is included in the top 2 results but non-relevant information is also included in the context. This is an indication that we need to improve our retrieval strategy. One possible solution could be to increase the number of documents retrieved and then use a more sophisticated ranking strategy (such as a reranker) to select the correct context.

We have now evaluated our RAG system's retrieval performance. Let's send these evaluations to Phoenix for visualization. By sending the evaluations to Phoenix, you will be able to view the evaluations alongside the traces that were captured earlier.

In [36]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=ndcg_at_2, eval_name="ndcg@2"),
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

### Response Evaluation

The retrieval evaluations demonstrates that our RAG system is not perfect. However, it's possible that the LLM is able to generate the correct response even when the context is incorrect. Let's evaluate the responses generated by the LLM.

In [37]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.Client())
qa_with_reference_df

,input,output,reference
context.span_id,,,
a4341236d20a5b8e,What did the author do growing up?,The author focused on writing short stories an...,What I Worked On\n\nFebruary 2021\n\nBefore co...
b076540f57a2d69f,What were the two main things the author worke...,The author worked on writing and programming b...,What I Worked On\n\nFebruary 2021\n\nBefore co...
427f05c395c72250,Explain the author's realization about AI duri...,The author realized during their first year of...,"I applied to 3 grad schools: MIT and Yale, whi..."
28d151a33d75cb72,What were the two main projects the author was...,Writing essays and working on Y Combinator.,Now lots of startups get their initial set of ...
5980c087b7d0d7c5,How did the narrator's return to New York impa...,The narrator's return to New York positively i...,"That's what I should have done, just gone off ..."
c78565c67473f80d,How does the process of painting still lives d...,The process of painting still lives allows for...,Our model turned out to live just down the str...
02827178feb7ae7d,How did the author's experience with computers...,The author's experience with computers involve...,I remember vividly how impressed and envious I...
6ee3ad3085fa1a79,Why did the author decide to name the new comp...,The author decided to name the new company 'As...,This name didn't last long before it was repla...
e0dbf7cf454206cf,Why did the founders of YC decide to recruit S...,The founders of YC decided to recruit Sam Altm...,"I asked Jessica if she wanted to be president,..."


Now that we have a dataset of the question, context, and response (input, reference, and output), we now can measure how well the LLM is responding to the queries. For details on the QA correctness evaluation, see the [LLM Evals documentation](https://docs.arize.com/phoenix/llm-evals/running-pre-tested-evals/q-and-a-on-retrieved-data).

In [38]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    run_evals,
)

qa_evaluator = QAEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))
hallucination_evaluator = HallucinationEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))

qa_correctness_eval_df, hallucination_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

run_evals |          | 0/62 (0.0%) | ⏳ 00:00<? | ?it/s

In [39]:
qa_correctness_eval_df.head()

,label,score,explanation
context.span_id,,,
a4341236d20a5b8e,correct,1.0,The reference text provides detailed informati...
b076540f57a2d69f,correct,1.0,"To determine if the answer is correct, we comp..."
427f05c395c72250,correct,1.0,The answer correctly summarizes the author's r...
28d151a33d75cb72,correct,1.0,The reference text clearly states that the aut...
5980c087b7d0d7c5,correct,1.0,The reference text indicates that the narrator...


In [40]:
hallucination_eval_df.head()

,label,score,explanation
context.span_id,,,
a4341236d20a5b8e,factual,0,The answer is factual because it accurately su...
b076540f57a2d69f,factual,0,The reference text explicitly states that befo...
427f05c395c72250,factual,0,The answer accurately summarizes the author's ...
28d151a33d75cb72,factual,0,The reference text explicitly states that the ...
5980c087b7d0d7c5,factual,0,The reference text provides information that s...


#### Observations

Let's now take our results and aggregate them to get a sense of how well the LLM is answering the questions given the context.

In [41]:
qa_correctness_eval_df.mean(numeric_only=True)

score    0.83871
dtype: float64

In [42]:
hallucination_eval_df.mean(numeric_only=True)

score    0.064516
dtype: float64

Our QA Correctness score of `0.91` and a Hallucinations score `0.05` signifies that the generated answers are correct ~91% of the time and that the responses contain hallucinations 5% of the time - there is room for improvement. This could be due to the retrieval strategy or the LLM itself. We will need to investigate further to determine the root cause.

Since we have evaluated our RAG system's QA performance and Hallucinations performance, let's send these evaluations to Phoenix for visualization.

In [43]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
)

We now have sent all our evaluations to Phoenix. Let's go to the Phoenix application and view the results! Since we've sent all the evals to Phoenix, we can analyze the results together to make a determination on whether or not poor retrieval or irrelevant context has an effect on the LLM's ability to generate the correct response.

In [44]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


## Conclusion

We have explored how to build and evaluate a RAG pipeline using LlamaIndex and Phoenix, with a specific focus on evaluating the retrieval system and generated responses within the pipelines. 

Phoenix offers a variety of other evaluations that can be used to assess the performance of your LLM Application. For more details, see the [LLM Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) documentation.